In [3]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
import numpy as np
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from autogluon.common import space
from sklearn.metrics import mean_absolute_error
from typing import List, Dict
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_regression
import numpy as np
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_autogluon
importlib.reload(preprocesamiento)
importlib.reload(model_autogluon)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


# Experimento 6: 
- Autogluon: FE automatizado
   

- Target: mes+2


- Kaggle =  0.336


In [9]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
df = df.merge(productos, on='product_id', how='left')
df = df[df['periodo']<201911]
df.isna().sum()

product_id                   0
periodo                      0
nacimiento_producto          0
muerte_producto              0
mes_n                        0
total_meses                  0
producto_nuevo               0
ciclo_de_vida_inicial        0
cat1_x                     254
cat2_x                     254
cat3_x                     254
brand_x                    254
sku_size_x                 254
stock_final              17782
tn                           0
plan_precios_cuidados      272
cust_request_qty           272
cust_request_tn            272
target                     538
cat1_y                     254
cat2_y                     254
cat3_y                     254
brand_y                    254
sku_size_y                 254
dtype: int64

In [ ]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')

df = feature_engineering.get_dolar(df)
df.shape

(31362, 20)

In [11]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
dfg = df.groupby(['periodo', 'product_id']).agg({'target': 'sum'}).reset_index()
dfg['timestamp'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
dfg.rename(columns={'product_id': 'item_id'}, inplace=True)

# Filtrar productos
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
dfg = dfg[dfg['item_id'].isin(productos_ok['product_id'].unique())]

# Convertir a TimeSeriesDataFrame
data = TimeSeriesDataFrame.from_data_frame(
    dfg,
    id_column="item_id",
    timestamp_column="timestamp"
)

In [20]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

def prepare_data(path="./datasets/periodo_x_producto_con_target.csv"):
    # Cargar datos
    df = pd.read_csv(path, sep=",", encoding="utf-8")
    
    # Convertir a timestamp
    df['timestamp'] = pd.to_datetime(df['periodo'].astype(str), format="%Y%m", errors='coerce')
    df = df.dropna(subset=['timestamp', 'target'])
    
    # Renombrar columnas para AutoGluon
    df = df.rename(columns={"product_id": "item_id"})
    
    # Validaciones mínimas
    if df['item_id'].nunique() < 1:
        raise ValueError("No hay suficientes series para entrenar.")
    if df['timestamp'].isnull().any():
        raise ValueError("Timestamps inválidos detectados.")
    
    return df

def add_lags(df, lags=[1, 2, 3]):
    """Agrega lags de la variable tn como features"""
    df = df.sort_values(['item_id', 'timestamp'])
    for lag in lags:
        df[f'lag_tn_{lag}'] = df.groupby('item_id')['tn'].shift(lag)
    return df

def run_autogluon_pipeline():
    df = prepare_data()
    df = add_lags(df)

    # Seleccionar columnas relevantes
    feature_cols = [col for col in df.columns if col.startswith('lag_tn_')]
    all_cols = ['item_id', 'timestamp', 'target'] + feature_cols
    df = df[all_cols].dropna()

    # 🔧 Filtrar series demasiado cortas
    series_lengths = df.groupby("item_id").size()
    items_ok = series_lengths[series_lengths > 1].index
    df = df[df['item_id'].isin(items_ok)]

    # Crear TimeSeriesDataFrame
    ts_df = TimeSeriesDataFrame.from_data_frame(
        df,
        id_column="item_id",
        timestamp_column="timestamp"
    )

    # Entrenar modelo
    predictor = TimeSeriesPredictor(
        prediction_length=1,  # Porque target ya es mes+2
        target="target",
        freq="M",
        eval_metric="MAPE",
    )

    predictor.fit(
        ts_df,
        time_limit=300,  # segundos
        num_val_windows=5,
        verbosity=2,
    )

    # 9. Generar predicciones
    predicciones = predictor.predict(ts_df)

    # Evaluar en el mismo dataset (podés usar holdout mejor)
    performance = predictor.evaluate(ts_df)
    print("📊 Performance:", performance)

    return predicciones

if __name__ == "__main__":
    predictor = run_autogluon_pipeline()


Frequency 'M' stored as 'ME'
Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\entregable\AutogluonModels\ag-20250629_185412'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       2.17 GB / 15.89 GB (13.7%)
Disk Space Avail:   413.56 GB / 893.49 GB (46.3%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 5,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 300,
 'verbosity': 2}

train_d

📊 Performance: {'MAPE': -17.25357465627524}


In [23]:
predicciones = predictor.copy()
predicciones = predicciones.reset_index()
predicciones.rename(columns={'item_id': 'product_id'}, inplace=True)
predicciones = predicciones[['product_id', 'mean']]
predicciones.to_csv('./outputs/predicciones_exp_06_autogluon_v6.csv', index=False, sep=',', encoding='utf-8')

In [25]:
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
predicciones = predicciones[predicciones['product_id'].isin(productos_ok['product_id'].unique())]
predicciones

,product_id,mean
0,20001,1477.161573
1,20002,1085.026099
2,20003,879.127719
3,20004,632.324335
4,20005,586.334104
...,...,...
1090,21263,0.007857
1092,21265,0.057972
1093,21266,0.059661
1094,21267,0.011926


In [26]:
output = pd.read_csv("./outputs/predicciones_exp_06_autogluon_v1_12kfold.csv", sep=',', encoding='utf-8')
output

,product_id,pred_windows_12
0,20001,1458.384452
1,20002,1193.750891
2,20003,771.721654
3,20004,561.147217
4,20005,602.650778
...,...,...
775,21263,0.023488
776,21265,0.096101
777,21266,0.093510
778,21267,0.074795


In [27]:
# Unión externa (outer join) para incluir todos los product_id
predicciones_completo = predicciones.merge(
    output[["product_id", "pred_windows_12"]],
    on="product_id",
    how="outer"
)

# Opcional: Ordenar por product_id
predicciones_completo = predicciones_completo.sort_values("product_id").reset_index(drop=True)

In [33]:
predicciones_completo

,product_id,mean,pred_windows_12
0,20001,1477.161573,1458.384452
1,20002,1085.026099,1193.750891
2,20003,879.127719,771.721654
3,20004,632.324335,561.147217
4,20005,586.334104,602.650778
...,...,...,...
775,21263,0.007857,0.023488
776,21265,0.057972,0.096101
777,21266,0.059661,0.093510
778,21267,0.011926,0.074795


In [37]:
# Rellenar pred_ponderado con pred_windows_12 donde pred_ponderado sea NaN (nuevos registros)
predicciones_completo["mean"] = predicciones_completo["mean"].fillna(
    predicciones_completo["pred_windows_12"]
)
predicciones_completo.isna().sum()

product_id         0
mean               0
pred_windows_12    0
dtype: int64

In [38]:
predicciones_completo.drop(columns=['pred_windows_12'], inplace=True)
predicciones_completo.rename(columns={'mean': 'tn'}, inplace=True)
predicciones_completo.to_csv('./outputs/predicciones_exp_06_autogluon_v6.csv', index=False, sep=',', encoding='utf-8')

In [39]:
predicciones_completo

,product_id,tn
0,20001,1477.161573
1,20002,1085.026099
2,20003,879.127719
3,20004,632.324335
4,20005,586.334104
...,...,...
775,21263,0.007857
776,21265,0.057972
777,21266,0.059661
778,21267,0.011926
